En este ejercicio queremos programar un sistema que recomiende
textos a usuarios en base a sus gustos sobre ciertos términos (palabras).
Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
es un string de longitud variable. Además contamos con un RDD que
indica qué términos le gustan o no a cada usuario de la forma (userId,
término, score) por ejemplo (23, “calesita”, -2). Se pide programar en
Spark un programa que calcule el score total de cada documento para
cada usuario generando un RDD de la forma (userId, docId, score) en
donde el score es simplemente la suma de los scores del usuario para
los términos que aparecen en el documento. Puede haber términos en
los documentos para los cuales no exista score de algunos usuarios, en
estos casos simplemente los consideramos neutros (score=0).  
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqs33v4xw)

Para empezar, definiremos un set de documentos y usuarios para poder hacer pruebas sobre nuestra resolucion.

In [ ]:
Quijote = 'En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino.'
Dickens = 'Las noches aqui son largas algunas veces, muy largas; pero nada son en comparacion con las inquietas noches y terribles ensueños de aquel tiempo. Su recuerdo me estremece. Grandes, sombríos fantasmas con maliciosos rostros se sentaban en los rincones de mi cuarto,y de noche se inclinaban sobre mi, incitándome á la locura. Me decian en voces atronadoras que el suelo de la antigua casa en que murió el padre de mi padre, estaba aun manchado de su sangre, derramada por su propia mano en el furor de su locura. Me cubria los oidos con las manos, pero me gritaban y me gritaban hasta que el cuarto se estremecia con sus acentos, y por todas partes oia que en la generacion anterior á la suya la locura durmió, pero que su abuelo habia vivido por años con sus manos entre grillos, para evitar que se hiciesen pedazos. Sabia que decian la verdad, lo sabia bien. Lo habia descubierto años hacia , aunque me lo quisieron ocultar.'
Napoleon = 'Nacido Napoleone di Buonaparte (Nabolione o Nabulione en corso), sólo un año después de que Francia comprara la isla de Córcega a la República de Génova.Napoleone, años después, cambió su nombre por el afrancesado Napoléon Bonaparte. El registro más antiguo de este nombre aparece en un informe oficial fechado el 28 de marzo de 1796. Su familia formaba parte de la nobleza local. Su padre, Carlo Buonaparte, abogado, fue nombrado en 1778 representante de Córcega en la corte de Luis XVI, lugar donde permaneció por varios años, por lo que fue su madre, María Letizia Ramolino, la figura fundamental de su niñez'
Agua = 'El agua (del latín aqua) es una sustancia cuya molécula está formada por dos átomos de hidrógeno y uno de oxígeno (H2O). Es esencial para la supervivencia de todas las formas conocidas de vida. El término agua generalmente se refiere a la sustancia en su estado líquido, aunque la misma puede hallarse en su forma sólida llamada hielo, y en su forma gaseosa denominada vapor. El agua cubre el 71% de la superficie de la corteza terrestre.2 Se localiza principalmente en los océanos donde se concentra el 96,5% del agua total, los glaciares y casquetes polares poseen el 1,74%, los depósitos subterráneos (acuíferos), los permafrost y los glaciares continentales suponen el 1,72% y el restante 0,04% se reparte en orden decreciente entre lagos, humedad del suelo, atmósfera, embalses, ríos y seres vivos.'
Politica = 'La política \(del latín politicus y esta del griego antiguo πολιτικός civil, relativo al ordenamiento de la ciudad o los asuntos del ciudadano\) es una rama de la moral que se ocupa de la actividad, en virtud de la cual una sociedad libre, compuesta por hombres libres, resuelve los problemas que le plantea su convivencia colectiva. Es un quehacer ordenado al bien común. Algunos autores presentan al uso legítimo de la fuerza como la característica principal de la política. Siguiendo con esta definición la política es el ejercicio del poder que busca un fin trascendente. Esta promueve la participación ciudadana ya que posee la capacidad de distribuir y ejecutar el poder según sea necesario para promover el bien común.'
Sintaxis = 'El orden de las palabras en una oración (sintaxis), la puntuación y la acentuación pueden variar según la intención del autor. En ciertas oraciones ello dependerá de que se emita una idea u otra diferente; en ocasiones lo opuesto a lo que se pretende expresar. Antes de empezar a escribir es necesario alinear mentalmente las ideas que se quieren trasladar al papel o al monitor o pantalla de la computadora u ordenador. Una vez ordenadas en la mente, es necesario identificar las ideas principales y las secundarias. Se elabora un esquema en el que se escriban en orden, según la importancia de cada una. Es importante el orden de una oración, ya que de lo contrario resultará carente de sentido. Más aún, si de manera lógica no se es capaz de ordenar lo que se quiere decir, el escrito perderá todo interés. Toda redacción necesita coherencia y cohesión textuales. En términos sencillos redactar es poner ideas en orden pero bien escritas.Algunos de los denominados «medios» cuentan con manuales de estilo, o imparten cursos de capacitación en sus «instalaciones». Personal que labora en empresas que no disponen de estas «facilidades» y que desea superarse toma cursos en instituciones públicas o privadas.'

Documentos = sc.parallelize([(1,Quijote),(2,Dickens),(3,Napoleon),(4,Agua),(5,Politica),(6,Sintaxis)])

PuntajesUser1 = [(1,"antiguo",2),(1,"Buonaparte",1),(1,"Francia",1)]
PuntajesUser2 = [(2,"en",-1),(2,"orden",4)]
PuntajesUser3 = [(3,"no",0)]

Puntajes = sc.parallelize([PuntajesUser1,PuntajesUser2,PuntajesUser3])

En primer lugar vamos a separar los documentos en palabras y contar la cantidad de repeticiones. Nos queda algo del estilo (docId, palabra, repeticiones). Luego simplemente reordenamos para poder tener las palabras como clave de la tupla.
Luego tomamos el rdd de puntajes y lo acomodamos de modo de tener las palabras como clave. 
Teniendo estos dos RDDs acomodados, podemos hacer un join por palabras. Nos queda un RDD con valores del estilo (palabra,(docId,repeticiones),(userId,puntaje)).

In [ ]:
Palabras = Documentos.flatMap(lambda x: map(lambda y: ((y,x[0]),1),x[1].split())).reduceByKey(lambda x,y: x+y)
Palabras = Palabras.map(lambda x: (x[0][0],(x[0][1],x[1])))
Puntos = Puntajes.flatMap(lambda x: x).map(lambda x: (x[1],(x[0],x[2])))
Scores = Palabras.join(Puntos)
Scores.take(1)

Pero las palabras ya no nos importan. Nos importa el score de cada palabra por usuario y documento. Asi que primero armamos eso, con (userId,docId) como clave, para poder hacer un reduceByKey luego que sume los puntajes.
Luego de ello, solo le damos el formato pedido.

In [ ]:
Scores = Scores.map(lambda x: x[1]).map(lambda x: ((x[1][0],x[0][0]),x[0][1]*x[1][1]))
Scores = Scores.reduceByKey(lambda x,y: x+y).map(lambda x: x[0]+(x[1],))
Scores.collect()